In [1]:
!git clone https://github.com/ucbdrive/hd3.git

Cloning into 'hd3'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 106 (delta 43), reused 85 (delta 31), pack-reused 0
Receiving objects: 100% (106/106), 2.24 MiB | 4.37 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [2]:
%cd hd3

/content/hd3


In [3]:
!pip install -r /content/hd3/requirement.txt

     |████████████████████████████████| 194kB 9.3MB/s 
     |████████████████████████████████| 3.1MB 68.5MB/s 
  Created wheel for cupy: filename=cupy-6.5.0-cp36-cp36m-linux_x86_64.whl size=26510249 sha256=6d8ed1d8fa15b84b74bf648b1745aabb5e75a57287815eae24fb246b3d689e3c
  Stored in directory: /root/.cache/pip/wheels/d6/bd/4e/ce32a2bd6af01b2823b2335c3b842d5f2c2fe4a19f85627f27
Successfully built cupy


In [4]:
%%bash
#!/bin/bash
SAVE_DIR=model_zoo
mkdir -p ${SAVE_DIR}
webroot='dl.yf.io/hd3/models/'
model_names=(
hd3f_chairs-04bf114d.pth
hd3f_chairs_things-462a3896.pth
hd3f_chairs_things_kitti-41b15827.pth
hd3f_chairs_things_sintel-5b4ad51a.pth
hd3fc_chairs-1367436d.pth
hd3fc_chairs_things-0b92a7f5.pth
hd3fc_chairs_things_kitti-bfa97911.pth
hd3fc_chairs_things_sintel-0b6e4b67.pth
hd3s_things-8b4dcd6d.pth
hd3s_things_kitti-1243813e.pth
hd3sc_things-57947496.pth
hd3sc_things_kitti-368975c0.pth
)

for i in ${model_names[@]}; do
	model_url=${webroot}$i
	out_file=${SAVE_DIR}/$i
	echo "Downloading: "$model_url
	wget -N $model_url -O $out_file
done


Downloading: dl.yf.io/hd3/models/hd3f_chairs-04bf114d.pth
Downloading: dl.yf.io/hd3/models/hd3f_chairs_things-462a3896.pth
Downloading: dl.yf.io/hd3/models/hd3f_chairs_things_kitti-41b15827.pth
Downloading: dl.yf.io/hd3/models/hd3f_chairs_things_sintel-5b4ad51a.pth
Downloading: dl.yf.io/hd3/models/hd3fc_chairs-1367436d.pth
Downloading: dl.yf.io/hd3/models/hd3fc_chairs_things-0b92a7f5.pth
Downloading: dl.yf.io/hd3/models/hd3fc_chairs_things_kitti-bfa97911.pth
Downloading: dl.yf.io/hd3/models/hd3fc_chairs_things_sintel-0b6e4b67.pth
Downloading: dl.yf.io/hd3/models/hd3s_things-8b4dcd6d.pth
Downloading: dl.yf.io/hd3/models/hd3s_things_kitti-1243813e.pth
Downloading: dl.yf.io/hd3/models/hd3sc_things-57947496.pth
Downloading: dl.yf.io/hd3/models/hd3sc_things_kitti-368975c0.pth


for details.

--2019-11-22 22:14:50--  http://dl.yf.io/hd3/models/hd3f_chairs-04bf114d.pth
Resolving dl.yf.io (dl.yf.io)... 128.32.162.150
Connecting to dl.yf.io (dl.yf.io)|128.32.162.150|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 158551235 (151M) [application/octet-stream]
Saving to: ‘model_zoo/hd3f_chairs-04bf114d.pth’

     0K .......... .......... .......... .......... ..........  0%  175K 14m44s
    50K .......... .......... .......... .......... ..........  0%  349K 11m3s
   100K .......... .......... .......... .......... ..........  0% 65.7M 7m23s
   150K .......... .......... .......... .......... ..........  0% 94.5M 5m32s
   200K .......... .......... .......... .......... ..........  0%  352K 5m54s
   250K .......... .......... .......... .......... ..........  0%  105M 4m55s
   300K .......... .......... .......... .......... ..........  0%  351K 5m16s
   350K .......... .......... .......... .......... ..........  0% 61.6M 4m36s
   400K .....

In [5]:
%%bash
mkdir custom_data
mkdir lists
touch lists/test_data_list.txt
mkdir out
cp /content/000020_left.jpg custom_data/000020_left.jpg
cp /content/000020_right.jpg custom_data/000020_right.jpg
# and some manual steps inputing the list

mkdir: cannot create directory ‘lists’: File exists


In [15]:
%%bash
python -u inference.py \
  --task=stereo \
  --data_root=custom_data \
  --data_list=lists/test_data_list.txt \
  --context \
  --encoder=dlaup \
  --decoder=hda \
  --batch_size=1 \
  --workers=16 \
  --flow_format=png \
  --context \
  --model_path=/content/hd3/model_zoo/hd3sc_things_kitti-368975c0.pth \
  --save_folder=out


[2019-11-22 22:32:36,436 INFO inference.py line 78 1291] Namespace(batch_size=1, context=True, data_list='lists/test_data_list.txt', data_root='custom_data', decoder='hda', encoder='dlaup', evaluate=False, flow_format='png', model_path='/content/hd3/model_zoo/hd3sc_things_kitti-368975c0.pth', save_folder='out', task='stereo', workers=16)
[2019-11-22 22:32:36,436 INFO inference.py line 79 1291] => creating model ...
[2019-11-22 22:32:41,311 INFO inference.py line 121 1291] HD3Model(
  (hd3net): HD3Net(
    (encoder): DLAUpEncoder(
      (base): DLA(
        (base_layer): Sequential(
          (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
          (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (level0): Sequential(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, a